In [ ]:
import os
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
import threading
from threading import Thread, Lock
import time
import tensorflow_probability as tfp
from typing import Any, List, Sequence, Tuple

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
          [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

tfd = tfp.distributions

class OurModel(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(OurModel, self).__init__()
        
        self.flatten = Flatten()
        self.dense_0 = Dense(512, activation='relu')
        self.dense_1 = Dense(action_space)
        self.dense_2 = Dense(1)
        
    def call(self, X_input):
        X_input = self.flatten(X_input)
        X_input = self.dense_0(X_input)
        action_logit = self.dense_1(X_input)
        value = self.dense_2(X_input)
        
        return action_logit, value


def safe_log(x):
  """Computes a safe logarithm which returns 0 if x is zero."""
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    """
    For a batch of vectors, take a single vector component
    out of each vector.
    Args:
      vectors: a [batch x dims] Tensor.
      indices: an int32 Tensor with `batch` entries.
    Returns:
      A Tensor with `batch` entries, one for each vector.
    """
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))


huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
sparse_ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
mse_loss = tf.keras.losses.MeanSquaredError()


class IMPALA_Agent:
    # IMPALA Main Optimization Algorithm
    def __init__(self, env_name):
        # Initialization Environment and parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.action_size = self.env.action_space.n
        self.EPISODES, self.episode, self.max_average = 2000000, 0, -21.0 # specific for pong
        
        self.memory_size = 50000
        self.memory = []
        self.lock = Lock()
        self.lr = 0.0001

        num_hidden_units = 512
    
        self.batch_size = 32
        self.ROWS = 80
        self.COLS = 80
        self.REM_STEP = 4
        
        self.state_size = (self.COLS, self.ROWS, self.REM_STEP)
        self.image_memory = np.zeros(self.state_size)
        
        # Instantiate plot memory
        self.scores, self.episodes, self.average = [], [], []

        self.Save_Path = 'Models'
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_IMPALA_{}'.format(self.env_name, self.lr)
        self.model_name = os.path.join(self.Save_Path, self.path)

        # Create Actor-Critic network model
        self.model = OurModel(input_shape=self.state_size, action_space=self.action_size)
        
        self.optimizer = tf.keras.optimizers.Adam(self.lr)

    def remember(self, state, action, policy, reward, done):
        experience = state, action, policy, reward, done
        if len(self.memory) <= self.memory_size:
            self.memory.append((experience))
        else:
            self.memory = []
            
    def act(self, state):
        prediction = self.model(state, training=False)
        dist = tfd.Categorical(logits=prediction[0])
        action = int(dist.sample()[0])
        policy = prediction[0]
        
        return action, policy

    def update(self, states, actions, agent_policies, rewards, dones):
        online_variables = self.model.trainable_variables
        with tf.GradientTape() as tape:
            tape.watch(online_variables)
            
            learner_outputs = self.model(states, training=True)
            
            agent_logits = tf.nn.softmax(agent_policies[:-1])
            actions = actions[:-1]
            rewards = rewards[1:]
            dones = dones[1:]
        
            learner_policies = learner_outputs[0]
            learner_logits = tf.nn.softmax(learner_policies[:-1])
            
            learner_values = learner_outputs[1]
            learner_values = tf.squeeze(learner_values)
            
            bootstrap_value = learner_values[-1]
            learner_values = learner_values[:-1]
            
            discounting = 0.99
            discounts = tf.cast(~dones, tf.float32) * discounting
            
            actions = tf.convert_to_tensor(actions, dtype=tf.int32)
                
            target_action_probs = take_vector_elements(learner_logits, actions)
            target_action_log_probs = tf.math.log(target_action_probs)
            
            behaviour_action_probs = take_vector_elements(agent_logits, actions)
            behaviour_action_log_probs = tf.math.log(behaviour_action_probs)
            
            lambda_ = 1.0
            
            log_rhos = target_action_log_probs - behaviour_action_log_probs
            
            log_rhos = tf.convert_to_tensor(log_rhos, dtype=tf.float32)
            discounts = tf.convert_to_tensor(discounts, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            values = tf.convert_to_tensor(learner_values, dtype=tf.float32)
            bootstrap_value = tf.convert_to_tensor(bootstrap_value, dtype=tf.float32)
            
            clip_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            clip_pg_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            
            rhos = tf.math.exp(log_rhos)
            
            clipped_rhos = tf.minimum(clip_rho_threshold, rhos, name='clipped_rhos')
            
            cs = tf.minimum(1.0, rhos, name='cs')
            cs *= tf.convert_to_tensor(lambda_, dtype=tf.float32)

            values_t_plus_1 = tf.concat([values[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            deltas = clipped_rhos * (rewards + discounts * values_t_plus_1 - values)
        
            acc = tf.zeros_like(bootstrap_value)
            vs_minus_v_xs = []
            for i in range(int(discounts.shape[0]) - 1, -1, -1):
                discount, c, delta = discounts[i], cs[i], deltas[i]
                acc = delta + discount * c * acc
                vs_minus_v_xs.append(acc)  
            
            vs_minus_v_xs = vs_minus_v_xs[::-1]
            
            vs = tf.add(vs_minus_v_xs, values, name='vs')
            vs_t_plus_1 = tf.concat([vs[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            clipped_pg_rhos = tf.minimum(clip_pg_rho_threshold, rhos, name='clipped_pg_rhos')
            
            pg_advantages = (clipped_pg_rhos * (rewards + discounts * vs_t_plus_1 - values))
            
            vs = tf.stop_gradient(vs)
            pg_advantages = tf.stop_gradient(pg_advantages)
            
            actor_loss = -tf.reduce_mean(target_action_log_probs * pg_advantages)
            
            baseline_cost = 0.5
            v_error = values - vs
            critic_loss = baseline_cost * 0.5 * tf.reduce_mean(tf.square(v_error))
            
            total_loss = actor_loss + critic_loss

        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
    
    def replay(self):
        memory_len = len(self.memory)
        if len(self.memory) > self.batch_size:
            start_index = random.randint(0, memory_len - self.batch_size)
            minibatch = self.memory[start_index:start_index+self.batch_size]
        else:
            return

        states = np.zeros((self.batch_size, *self.state_size), dtype=np.float32)
        actions = np.zeros(self.batch_size, dtype=np.int32)
        policies = np.zeros((self.batch_size, self.action_size), dtype=np.float32)
        rewards = np.zeros(self.batch_size, dtype=np.float32)
        dones = np.zeros(self.batch_size, dtype=np.bool)
        for i in range(len(minibatch)):
            states[i] = minibatch[i][0]
            actions[i] = minibatch[i][1]
            policies[i] = minibatch[i][2]
            rewards[i] = minibatch[i][3]
            dones[i] = minibatch[i][4]
            
        self.update(states, actions, policies, rewards, dones)
        
    def load(self, model_name):
        self.model = load_model(model_name, compile=False)

    def save(self):
        self.model.save(self.model_name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes, self.scores, 'b')
            pylab.plot(self.episodes, self.average, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.savefig(self.path + ".png")
            except OSError:
                pass

        return self.average[-1]
    
    def imshow(self, image, rem_step=0):
        #print("image[:,:,rem_step].shape: ", image[:,:,rem_step].shape)
        
        cv2.imshow("pong" + str(rem_step), image[:,:,rem_step])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()

    def GetImage(self, frame):
        #print("frame.shape: ", frame.shape)
        
        # croping frame to 80x80 size
        frame_cropped = frame[35:195:2, ::2,:]
        if frame_cropped.shape[0] != self.COLS or frame_cropped.shape[1] != self.ROWS:
            # OpenCV resize function 
            frame_cropped = cv2.resize(frame, (self.COLS, self.ROWS), interpolation=cv2.INTER_CUBIC)
        
        # converting to RGB (numpy way)
        frame_rgb = 0.299*frame_cropped[:,:,0] + 0.587*frame_cropped[:,:,1] + 0.114*frame_cropped[:,:,2]
        
        # converting to Gray (OpenCV way)
        #frame_gray = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2GRAY)     
        #print("frame_gray.shape: ", frame_gray.shape)
        
        frame_rgb[frame_rgb < 100] = 0
        frame_rgb[frame_rgb >= 100] = 255
        # dividing by 255 we expresses value to 0-1 representation
        new_frame = np.array(frame_rgb).astype(np.float32) / 255.0

        # push our data by 1 frame, similar as deq() function work
        self.image_memory = np.roll(self.image_memory, 1, axis=2)

        # inserting new frame to free space
        self.image_memory[:,:,0] = new_frame

        # show image frame   
        #self.imshow(self.image_memory, 0)
        #self.imshow(self.image_memory, 1)
        #self.imshow(self.image_memory, 2)
        #self.imshow(self.image_memory, 3)

        return np.expand_dims(self.image_memory, axis=0)
        
    def reset(self, env):
        frame = env.reset()
        for i in range(self.REM_STEP):
            state = self.GetImage(frame)

        return state

    def step(self, action, env):
        next_state, reward, done, info = env.step(action)
        next_state = self.GetImage(next_state)
        
        return next_state, reward, done, info
    
    def train(self, n_threads):
        self.env.close()
        # Instantiate one environment per thread
        envs = [gym.make(self.env_name) for i in range(n_threads)]

        # Create threads
        threads = [threading.Thread(
                target=self.train_threading,
                daemon=True,
                args=(self, envs[i], i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
    
    def render(self, obs):
        cv2.imshow('obs', obs)
        cv2.waitKey(1)
    
    def train_threading(self, agent, env, thread):
        max_average = 15.0
        total_step = 0
        for e in range(self.EPISODES):
            state = self.reset(env)

            done = False
            score = 0
            SAVING = ''
            while not done:
                #self.env.render()
                
                action, policy = self.act(state)
                
                next_state, reward, done, _ = self.step(action, env)
                
                self.remember(state, action, policy, reward / 20.0, done)
                state = next_state
                score += reward
                if done:
                    break
                
                if thread == 0:
                    if total_step % 250 == 0:
                        # train model
                        self.replay()
                        #self.lock.release()
                    
                    total_step += 1
                
            # Update episode count
            with self.lock:
                average = self.PlotModel(score, self.episode)
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""
                
                print("total_step: ", total_step)
                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1
                 
    def test(self, Actor_name, Critic_name):
        self.load(Actor_name, Critic_name)
        for e in range(100):
            state = self.reset(self.env)
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state))
                state, reward, done, _ = self.step(action, self.env, state)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break

        self.env.close()


if __name__ == "__main__":
    env_name = 'Pong-v0'
    agent = IMPALA_Agent(env_name)
    
    #agent.run() # use as IMPALA
    agent.train(n_threads=2) # use as IMPALA
    #agent.test('Models/Pong-v0_A3C_2.5e-05_Actor.h5', '')

/home/kimbring2/.local/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
/home/kimbring2/.local/lib/python3.7/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()
2021-11-20 07:52:49.506080: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-20 07:52:50.248196: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2

total_step:  1189
episode: 0/2000000, thread: 0, score: -20.0, average: -20.00 SAVING
total_step:  0
episode: 1/2000000, thread: 1, score: -21.0, average: -20.50 
total_step:  2413
episode: 2/2000000, thread: 0, score: -20.0, average: -20.33 
total_step:  0
episode: 3/2000000, thread: 1, score: -21.0, average: -20.50 
total_step:  3724
episode: 4/2000000, thread: 0, score: -19.0, average: -20.20 
total_step:  0
episode: 5/2000000, thread: 1, score: -19.0, average: -20.00 SAVING
total_step:  4830
episode: 6/2000000, thread: 0, score: -21.0, average: -20.14 
total_step:  0
episode: 7/2000000, thread: 1, score: -21.0, average: -20.25 
total_step:  5959
episode: 8/2000000, thread: 0, score: -20.0, average: -20.22 
total_step:  0
episode: 9/2000000, thread: 1, score: -21.0, average: -20.30 
total_step:  7262
episode: 10/2000000, thread: 0, score: -20.0, average: -20.27 
total_step:  0
episode: 11/2000000, thread: 1, score: -20.0, average: -20.25 
total_step:  8571
episode: 12/2000000, threa

total_step:  62972
episode: 103/2000000, thread: 0, score: -21.0, average: -20.34 
total_step:  0
episode: 104/2000000, thread: 1, score: -21.0, average: -20.36 
total_step:  64073
episode: 105/2000000, thread: 0, score: -20.0, average: -20.38 
total_step:  0
episode: 106/2000000, thread: 1, score: -19.0, average: -20.36 
total_step:  65197
episode: 107/2000000, thread: 0, score: -21.0, average: -20.36 
total_step:  0
episode: 108/2000000, thread: 1, score: -20.0, average: -20.34 
total_step:  66358
episode: 109/2000000, thread: 0, score: -20.0, average: -20.36 
total_step:  0
episode: 110/2000000, thread: 1, score: -21.0, average: -20.36 
total_step:  67407
episode: 111/2000000, thread: 0, score: -21.0, average: -20.38 
total_step:  0
episode: 112/2000000, thread: 1, score: -21.0, average: -20.40 
total_step:  68490
episode: 113/2000000, thread: 0, score: -21.0, average: -20.40 
total_step:  0
episode: 114/2000000, thread: 1, score: -21.0, average: -20.40 
total_step:  69685
episode: 

total_step:  0
episode: 205/2000000, thread: 1, score: -21.0, average: -20.12 
total_step:  123952
episode: 206/2000000, thread: 0, score: -20.0, average: -20.12 
total_step:  0
episode: 207/2000000, thread: 1, score: -21.0, average: -20.12 
total_step:  125220
episode: 208/2000000, thread: 0, score: -21.0, average: -20.12 
total_step:  0
episode: 209/2000000, thread: 1, score: -21.0, average: -20.12 
total_step:  126331
episode: 210/2000000, thread: 0, score: -21.0, average: -20.12 
total_step:  0
episode: 211/2000000, thread: 1, score: -19.0, average: -20.10 
total_step:  127863
episode: 212/2000000, thread: 0, score: -19.0, average: -20.08 
total_step:  0
episode: 213/2000000, thread: 1, score: -19.0, average: -20.04 
total_step:  128976
episode: 214/2000000, thread: 0, score: -20.0, average: -20.04 
total_step:  0
episode: 215/2000000, thread: 1, score: -21.0, average: -20.04 
total_step:  130085
episode: 216/2000000, thread: 0, score: -21.0, average: -20.04 
total_step:  0
episode

total_step:  183942
episode: 306/2000000, thread: 0, score: -21.0, average: -20.18 
total_step:  0
episode: 307/2000000, thread: 1, score: -20.0, average: -20.16 
total_step:  185231
episode: 308/2000000, thread: 0, score: -20.0, average: -20.16 
total_step:  0
episode: 309/2000000, thread: 1, score: -20.0, average: -20.14 
total_step:  186338
episode: 310/2000000, thread: 0, score: -21.0, average: -20.20 
total_step:  0
episode: 311/2000000, thread: 1, score: -21.0, average: -20.28 
total_step:  187462
episode: 312/2000000, thread: 0, score: -20.0, average: -20.32 
total_step:  0
episode: 313/2000000, thread: 1, score: -21.0, average: -20.34 
total_step:  188698
episode: 314/2000000, thread: 0, score: -21.0, average: -20.40 
total_step:  0
episode: 315/2000000, thread: 1, score: -20.0, average: -20.38 
total_step:  190104
episode: 316/2000000, thread: 0, score: -21.0, average: -20.38 
total_step:  0
episode: 317/2000000, thread: 1, score: -21.0, average: -20.38 
total_step:  191239
ep

...

total_step:  0
episode: 1212/2000000, thread: 1, score: -21.0, average: -20.14 
total_step:  734107
episode: 1213/2000000, thread: 0, score: -21.0, average: -20.14 
total_step:  0
episode: 1214/2000000, thread: 1, score: -19.0, average: -20.12 
total_step:  735272
episode: 1215/2000000, thread: 0, score: -20.0, average: -20.12 
total_step:  0
episode: 1216/2000000, thread: 1, score: -20.0, average: -20.12 
total_step:  736682
episode: 1217/2000000, thread: 0, score: -19.0, average: -20.08 
total_step:  0
episode: 1218/2000000, thread: 1, score: -17.0, average: -20.04 
total_step:  738172
episode: 1219/2000000, thread: 0, score: -19.0, average: -20.02 
total_step:  0
episode: 1220/2000000, thread: 1, score: -20.0, average: -20.00 
total_step:  739232
episode: 1221/2000000, thread: 0, score: -21.0, average: -20.02 
total_step:  0
episode: 1222/2000000, thread: 1, score: -20.0, average: -20.00 
total_step:  740373
episode: 1223/2000000, thread: 0, score: -20.0, average: -20.02 
total_step

total_step:  795313
episode: 1312/2000000, thread: 0, score: -21.0, average: -20.26 
total_step:  0
episode: 1313/2000000, thread: 1, score: -20.0, average: -20.28 
total_step:  796415
episode: 1314/2000000, thread: 0, score: -21.0, average: -20.30 
total_step:  0
episode: 1315/2000000, thread: 1, score: -19.0, average: -20.26 
total_step:  797644
episode: 1316/2000000, thread: 0, score: -20.0, average: -20.30 
total_step:  0
episode: 1317/2000000, thread: 1, score: -21.0, average: -20.30 
total_step:  798829
episode: 1318/2000000, thread: 0, score: -21.0, average: -20.32 
total_step:  0
episode: 1319/2000000, thread: 1, score: -21.0, average: -20.34 
total_step:  800064
episode: 1320/2000000, thread: 0, score: -21.0, average: -20.36 
total_step:  0
episode: 1321/2000000, thread: 1, score: -19.0, average: -20.34 
total_step:  801364
episode: 1322/2000000, thread: 0, score: -19.0, average: -20.32 
total_step:  0
episode: 1323/2000000, thread: 1, score: -20.0, average: -20.32 
total_step

total_step:  856564
episode: 1412/2000000, thread: 0, score: -21.0, average: -20.12 
total_step:  0
episode: 1413/2000000, thread: 1, score: -20.0, average: -20.14 
total_step:  857664
episode: 1414/2000000, thread: 0, score: -21.0, average: -20.18 
total_step:  0
episode: 1415/2000000, thread: 1, score: -19.0, average: -20.14 
total_step:  858816
episode: 1416/2000000, thread: 0, score: -20.0, average: -20.12 
total_step:  0
episode: 1417/2000000, thread: 1, score: -19.0, average: -20.08 
total_step:  860102
episode: 1418/2000000, thread: 0, score: -21.0, average: -20.12 
total_step:  861149
episode: 1419/2000000, thread: 0, score: -21.0, average: -20.14 
total_step:  0
episode: 1420/2000000, thread: 1, score: -20.0, average: -20.12 
total_step:  0
episode: 1421/2000000, thread: 1, score: -20.0, average: -20.10 
total_step:  862492
episode: 1422/2000000, thread: 0, score: -21.0, average: -20.12 
total_step:  0
episode: 1423/2000000, thread: 1, score: -20.0, average: -20.12 
total_step

total_step:  0
episode: 1512/2000000, thread: 1, score: -21.0, average: -20.18 
total_step:  918514
episode: 1513/2000000, thread: 0, score: -20.0, average: -20.18 
total_step:  0
episode: 1514/2000000, thread: 1, score: -21.0, average: -20.22 
total_step:  919902
episode: 1515/2000000, thread: 0, score: -18.0, average: -20.18 
total_step:  0
episode: 1516/2000000, thread: 1, score: -21.0, average: -20.18 
total_step:  0
episode: 1517/2000000, thread: 1, score: -20.0, average: -20.16 
total_step:  921301
episode: 1518/2000000, thread: 0, score: -19.0, average: -20.16 
total_step:  0
episode: 1519/2000000, thread: 1, score: -20.0, average: -20.16 
total_step:  922417
episode: 1520/2000000, thread: 0, score: -20.0, average: -20.14 
total_step:  0
episode: 1521/2000000, thread: 1, score: -20.0, average: -20.14 
total_step:  923991
episode: 1522/2000000, thread: 0, score: -20.0, average: -20.16 
total_step:  0
episode: 1523/2000000, thread: 1, score: -20.0, average: -20.16 
total_step:  92

total_step:  978732
episode: 1612/2000000, thread: 0, score: -21.0, average: -20.18 
total_step:  0
episode: 1613/2000000, thread: 1, score: -21.0, average: -20.18 
total_step:  979780
episode: 1614/2000000, thread: 0, score: -21.0, average: -20.24 
total_step:  0
episode: 1615/2000000, thread: 1, score: -21.0, average: -20.26 
total_step:  981111
episode: 1616/2000000, thread: 0, score: -21.0, average: -20.28 
total_step:  0
episode: 1617/2000000, thread: 1, score: -21.0, average: -20.30 
total_step:  0
episode: 1618/2000000, thread: 1, score: -20.0, average: -20.32 
total_step:  982401
episode: 1619/2000000, thread: 0, score: -20.0, average: -20.34 
total_step:  0
episode: 1620/2000000, thread: 1, score: -20.0, average: -20.32 
total_step:  983536
episode: 1621/2000000, thread: 0, score: -21.0, average: -20.34 
total_step:  984629
episode: 1622/2000000, thread: 0, score: -21.0, average: -20.34 
total_step:  0
episode: 1623/2000000, thread: 1, score: -20.0, average: -20.36 
total_step